## 5.6 Carsy

Aplikacja wykorzystuje `BottomNavigation` oraz `DrawerNavigation`. Po raz kolejny wykorzystamy tylko statyczne dostarczone przez obiekt `dataProvider`. Wykorzystamy `RecyclerView` w kilku różnych konfiguracjach. Będzie to uboga wersja [Fuelio](https://play.google.com/store/apps/details?id=com.kajda.fuelio&hl=pl&gl=US)
- `RecyclerView` z grupowaniem elementów osiągniętym przez wykorzystanie dwóch `ViewHolder`

### **BottomNavigation**

Rozpocznijmy od dodania trzech pustych fragmentów (`OverviewFragment`, `CalculatorsFragment`, `TimeLineFragment`) i nawigacji

`navigation.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/overviewFragment">

    <fragment
        android:id="@+id/calculatorsFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.CalculatorsFragment"
        android:label="fragment_calculators"
        tools:layout="@layout/fragment_calculators" />
    <fragment
        android:id="@+id/timeLineFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.TimeLineFragment"
        android:label="fragment_time_line"
        tools:layout="@layout/fragment_time_line" />
    <fragment
        android:id="@+id/overviewFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.OverviewFragment"
        android:label="fragment_overview"
        tools:layout="@layout/fragment_overview" />
</navigation>
```

Po raz kolejny dodajemy tylko fragmenty nie definiując dla nich akcji. `OverviewFragment` oznaczam jako fragment domowy. Musimy również utworzyć `menu` dla `BottomNavigation`

`bottom_menu.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/overviewFragment"
        android:title="@string/overview" />
    <item
        android:id="@id/timeLineFragment"
        android:title="@string/time_line" />

    <item
        android:id="@id/calculatorsFragment"
        android:title="@string/calculators" />
</menu>
```

Pamiętajmy, że `android:id` muszą być takie same jak w `navigation.xml` aby automatyczna nawigacja działała prawidłowo.

Przejdźmy do `MainActivity` i dodajmy nawigację oraz połączmy ją z `BottomNavigation`

In [ ]:
public class MainActivity extends AppCompatActivity {

    private NavController navController;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.nav_host_fragment);

        if (navHostFragment != null) {
            navController = NavHostFragment.findNavController(navHostFragment);
        }

        BottomNavigationView bottomNavigationView = findViewById(R.id.bottom_nav_view);
        NavigationUI.setupWithNavController(bottomNavigationView, navController);
    }
}

W efekcie powinniśmy otrzymać podstawową nawigację z trzema ekranami.

<img src="https://media1.giphy.com/media/x0j1SX0qmejSUNRtHu/giphy.gif?cid=790b7611bcaddd3fe77b775f20dd38e5e0cfa788c1e83df7&rid=giphy.gif&ct=g" width="150" />

### **TimeLineFragment**

Chcemy przygotować listę w postaci osi czasu, która będzie zbliżona do tego co oferuje aplikacja **Fuelio**

<img src="https://play-lh.googleusercontent.com/Lm6SO1jsUF-VjsrfkIV9x-qpPIkIOIvgWkBGmxyZuHWPSbkYMS0oLhgoBk9wLXq6Xw=w5120-h2880" width="150" />

Zrobimy to w postaci `RecyclerView` z dwoma `ViewHolder`. Wpierw przygotujmy dane. Typ kosztu zdefiniujemy jako `enum class`, będziemy przechowywać w niej typ oraz ikonę.

In [ ]:
public enum CostType {
    REFUELING("Tankowanie", R.drawable.ic_fuel),
    SERVICE("Serwis", R.drawable.ic_car_repair),
    PARKING("Parking", R.drawable.ic_parking),
    INSURANCE("Ubezpieczenie", R.drawable.ic_general_cost),
    TICKET("Mandat", R.drawable.ic_ticket);

    private final String costType;
    private final int icon;

    CostType(String costType, int icon) {
        this.costType = costType;
        this.icon = icon;
    }

    public String getCostType() {
        return costType;
    }

    public int getIcon() {
        return icon;
    }
}

Ikony standardowo wybieramy spośród dostępnych w **Android Studio** (**New -> Vector Asset**). Model danych zawiera typ kosztu, datę oraz kwotę.

In [ ]:
public class Cost {
    private final CostType type;
    private final LocalDate date;
    private final int amount;

    public Cost(CostType type, LocalDate date, int amount){
        this.type = type;
        this.date = date;
        this.amount = amount;
    }

    public CostType getType() {
        return type;
    }

    public LocalDate getDate() {
        return date;
    }

    public int getAmount() {
        return amount;
    }
}

Ponieważ będziemy posiadać pojedynczy `RecyclerView` i dwa `ViewHolder`, musimy w jakiś sposób rozróżnić typ naszych danych - dla uproszczenia będziemy sortować po miesiącach (bez uwzględnienia lat). Więc chcemy zdefiniować dwa typy 
- `CostListItem` - klasa bazowa

In [ ]:
public abstract class CostListItem {
    public static final int TYPE_DATE = 0;
    public static final int TYPE_GENERAL = 1;

    abstract public int getType();
}

- `CostDateItem` - posiadający tylko datę po której będziemy sortować oraz na podstawie której utworzymy wersję layoutu dla `RecyclerView`

In [ ]:
public class CostDateItem extends CostListItem {
    private String date;

    public CostDateItem(String date){
        this.date = date;
    }

    public String getDate() {
        return date;
    }

    @Override
    public int getType() {
        return TYPE_DATE;
    }
}

- `CostGeneralItem` - każdy inny element dla którego również utworzymy layout

In [ ]:
public class CostGeneralItem extends CostListItem {
    private Cost cost;

    public Cost getCost() {
        return cost;
    }

    public CostGeneralItem(Cost cost){
        this.cost = cost;
    }


    @Override
    public int getType() {
        return TYPE_GENERAL;
    }
}

Przejdźmy do obiektu `DataProvider` - w pierwszym kroku utworzymy listę kosztów

In [ ]:
public final class DataProvider {
    private DataProvider() {
    }

    public static ArrayList<Cost> getGeneralCosts() {
        ArrayList<Cost> items = new ArrayList<>();
        for (int i = 0; i < 100; i++) {
            items.add(new Cost(
                    CostType.values()[new Random().nextInt(CostType.values().length)],
                    LocalDate.of(2022, new Random().nextInt(12) + 1, new Random().nextInt(28) + 1),
                    new Random().nextInt(5000)));
        }
        return items;
    }
}

Ponieważ mamy jeden `RecyclerView`, więc będziemy mieć jedną listę - musimy ją odpowiednio przygotować. Wykorzystamy `SortedMap` i posortujemy całość po miesiącach, następnie pogrupujemy względem miesięcy.

In [ ]:
private static final SortedMap<Month, ArrayList<Cost>> groupedHashMap = 
    groupDataIntoHashMap(getGeneralCosts());

private static SortedMap<Month, ArrayList<Cost>> groupDataIntoHashMap(ArrayList<Cost> itemList) {

    SortedMap<Month, ArrayList<Cost>> groupedHashMap = new TreeMap<>();
    for (Cost cost : itemList) {
        Month hashMapKey = cost.getDate().getMonth();
        if (groupedHashMap.containsKey(hashMapKey)) {
            Objects.requireNonNull(groupedHashMap.get(hashMapKey)).add(cost);
        } else {
            ArrayList<Cost> list = new ArrayList<>();
            list.add(cost);
            groupedHashMap.put(hashMapKey, list);
        }
    }
    return groupedHashMap;
}

Następnie napiszemy funkcję `getTimeLineList` zwracająca listę dla `RecyclerView`

In [ ]:
public static ArrayList<CostListItem> getTimeLineList() {

Utwórzmy listę pomocniczą

In [ ]:
ArrayList<CostListItem> items = new ArrayList<>();

Przechodzimy po wszystkich kluczach w naszej mapie

In [ ]:
for (Month date : groupedHashMap.keySet()) {

W pętli tworzę listę wszystkich elementów z danego miesiąca, następnie ją sortuję względem dnia miesiąca i każdy element `CostGeneralItem` dodaję do `list`

In [ ]:
ArrayList<Cost> groupingItems = groupedHashMap.get(date);
if (groupingItems != null) {
    groupingItems.sort(Comparator.comparingInt(o -> o.getDate().getDayOfMonth()));
    groupingItems.forEach(cost -> items.add(new CostGeneralItem(cost)));
}

Ostatnim elementem pętli będzie dodanie `CostDateItem`

In [ ]:
CostDateItem dateItem = new CostDateItem(date.name());
items.add(dateItem);

Na koniec wychodzimy z pętli `for` i zwracamy listę

In [ ]:
    }
    return items;
}

Pełna funkcja

In [ ]:
public static ArrayList<CostListItem> getTimeLineList() {
    ArrayList<CostListItem> items = new ArrayList<>();

    for (Month date : groupedHashMap.keySet()) {
        ArrayList<Cost> groupingItems = groupedHashMap.get(date);
        if (groupingItems != null) {
            groupingItems.sort(Comparator.comparingInt(o -> o.getDate().getDayOfMonth()));
            groupingItems.forEach(cost -> items.add(new CostGeneralItem(cost)));
        }
        CostDateItem dateItem = new CostDateItem(date.name());
        items.add(dateItem);
    }
    return items;
}

Podsumowując:
- tworzą listę do której wrzucę wszystkie elementy odpowiednio posortowane - lista zawiera elementy typu `CostListItem`
- przechodzę po kluczach wcześniej utworzonej mapy (miesiące)
- dodaję wszystkie elementy `CostGeneralItem` - wszystkie koszty dla danego miesiąca
- na koniec dodaję `CostDateItem` - miesiąc

Przejdźmy do `TimeLineAdapter` - klasa będzie przyjmowała w parametrze `Context` i rozszerzała `RecyclerView.Adapter`

In [ ]:
public class TimeLineAdapter extends RecyclerView.Adapter<RecyclerView.ViewHolder> {}

W pierwszej kolejności dodajmy listę elementów.

In [ ]:
private final ArrayList<CostListItem> itemList = DataProvider.getTimeLineList();

Zdefiniujmy dwie klasy `ViewHolder` - pierwszą będzie klasa dla elementu grupującego (tutaj jest to data - miesiąc)

In [ ]:
    static class DateViewHolder extends RecyclerView.ViewHolder{

        private final TextView dateTextView;

        public DateViewHolder(@NonNull View itemView) {
            super(itemView);

            dateTextView = itemView.findViewById(R.id.timeLineDateTextView);
        }

        public void bind(CostDateItem item){
            dateTextView.setText(item.getDate());
        }
    }

Dla tego `ViewHolder` definiujemy layout w pliku `date_item_timeline.xml`


```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="19dp"
        android:layout_marginTop="24dp"
        android:background="@drawable/ic_round_circle"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <TextView
        android:id="@+id/timeLineDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="24dp"
        android:fontFamily="serif"
        android:textColor="?attr/colorSecondary"
        android:padding="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Jak widzimy mamy tylko jedno pole `TextView` wyświetlające nazwę miesiąca po którym grupujemy. Pozostałe dwa elementy są graficznym indykatorem - `View` jest linią o zadanej grubości i `ImageView` tutaj ma formę wypełnionego koła.

Zdefiniujmy również drugi `ViewHolder` dla pozostałych elementów na liście - tutaj chcemy pokazać typ kosztu, kwotę, pełną datę, oraz ikonę dla odpowiedniego typu kosztu.

In [ ]:
class GeneralViewHolder extends RecyclerView.ViewHolder{

    private final TextView costTypeTextView;
    private final TextView fullDateTextView;
    private final TextView amountTextView;
    private final ImageView iconImageView;

    public GeneralViewHolder(@NonNull View itemView) {
        super(itemView);

        costTypeTextView = itemView.findViewById(R.id.timeLineCostTypeNameTextView);
        fullDateTextView = itemView.findViewById(R.id.timeLineFullDateTextView);
        amountTextView = itemView.findViewById(R.id.timeLineCostAmountTextView);
        iconImageView = itemView.findViewById(R.id.iconTimeLineImageView);
    }

    public void bind(CostGeneralItem item){
        costTypeTextView.setText(item.getCost().getType().getCostType());
        fullDateTextView.setText(item.getCost().getDate().format(FormatterUtil.dateFormatter));
        amountTextView.setText(String.format("%s zł", item.getCost().getAmount()));
        iconImageView.setBackground(ContextCompat.getDrawable(context, item.getCost().getType().getIcon()));
    }
}

Będziemy również formatować nieco `String` przy dacie, więc zdefiniujmy `DateTimeFormatter`

In [ ]:
public final class FormatterUtil {
    private FormatterUtil(){}

    public static DateTimeFormatter dateFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd");
}

Layout jest nieco bardziej skomplikowany

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:id="@+id/imageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="3.2dp"
        android:background="@drawable/ic_round_circle_big"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <ImageView
        android:id="@+id/iconTimeLineImageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="12dp"
        android:layout_marginTop="8dp"
        android:background="@drawable/ic_car_repair"
        android:backgroundTint="@color/white"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/imageView"
        app:layout_constraintStart_toStartOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="@+id/imageView"
        app:layout_constraintVertical_bias="0.285" />

    <TextView
        android:id="@+id/timeLineCostTypeNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="4dp"
        android:text="@string/costtype_name"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineCostAmountTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="16dp"
        android:padding="4dp"
        android:text="1200 zł"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineFullDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="serif"
        android:paddingStart="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toBottomOf="@+id/timeLineCostTypeNameTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Zauważmy że znów mamy `View` o tych samych parametrch co w poprzednim layoucie co sprawia wrażenie ciągłości przez wszystkie elementy. Mamy również dwa `ImageView` - pierwszy zastosowałem jako okrągłe tło o jednolitym kolorze dla ikony obecnej w drugim.

Przejdźmy do metody `onCreateViewHolder` - zwrócimy `ViewHolder` dla zadanego `viewType`. Ponieważ będziemy wykorzystywać dwa, musimy zdefiniować jeszcze `viewType`.

In [ ]:
@NonNull
@Override
public RecyclerView.ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {

    RecyclerView.ViewHolder viewHolder;
    LayoutInflater inflater = LayoutInflater.from(parent.getContext());

    switch (viewType){
        case CostListItem.TYPE_DATE:
            View dateView = inflater.inflate(R.layout.date_item_timeline, parent, false);
            viewHolder = new DateViewHolder(dateView);
            break;
        case CostListItem.TYPE_GENERAL:
            View generalView = inflater.inflate(R.layout.general_item_timeline, parent, false);
            viewHolder = new GeneralViewHolder(generalView);
            break;
        default:
            throw new IllegalStateException("Unexpected value: " + viewType);
    }

    return viewHolder;
}

Aby zdefiniować `viewType` musimy nadpisać metodę `getItemViewType`

In [ ]:
public int getItemViewType(int position) {
    return itemList.get(position).getType();
} // zwraca typ elementu na liście

Zaimplementujmy metodę `getItemCount`

In [ ]:
@Override
public int getItemCount() {
    return itemList.size();
}

Pozostaje implementacja metody `onBindViewHolder` - w zależności od typu `ViewHolder` chcemy przekazać `CostListItem` jako `CostGeneralItem` zawierający instancję `Cost` lub `CostDateItem` zawierający nazwę miesiąca jako `String`.

In [ ]:
override fun onBindViewHolder(holder: RecyclerView.ViewHolder, position: Int) {
    when (holder.itemViewType) {
        CostListItem.TYPE_DATE -> (holder as DateViewHolder).bind(
            item = itemList[position] as CostDateItem
        )
        CostListItem.TYPE_GENERAL -> (holder as GeneralViewHolder).bind(
            item = itemList[position] as CostGeneralItem
        )
    }
}

Przejdźmy do `TimeLineFragment` i dodajmy nasz RecyclerView

In [ ]:
@Override
public void onBindViewHolder(@NonNull RecyclerView.ViewHolder holder, int position) {
    switch (holder.getItemViewType()){
        case CostListItem.TYPE_DATE:
            CostDateItem costDateItem = (CostDateItem) itemList.get(position);
            DateViewHolder dateViewHolder = (DateViewHolder) holder;
            dateViewHolder.bind(costDateItem);
            break;
        case CostListItem.TYPE_GENERAL:
            CostGeneralItem costGeneralItem = (CostGeneralItem) itemList.get(position);
            GeneralViewHolder generalViewHolder = (GeneralViewHolder) holder;
            generalViewHolder.bind(costGeneralItem);
            break;
        default:
            throw new IllegalStateException("Unexpected type");
    }
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/S4iOJk9GSVcizP2tRw/giphy.gif?cid=790b76113b3cee84bee0e3484c2fbc03807cf26e022c376a&rid=giphy.gif&ct=g" width="150" />